# Hello world attempt

## Installation of required packages

In [10]:
!pip install --upgrade diffusers accelerate transformers

SyntaxError: invalid syntax (1449779802.py, line 3)

## Accessing the pre-trained model

In [9]:
from diffusers import DiffusionPipeline

ModuleNotFoundError: No module named 'diffusers'

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)

In [ ]:
pipeline

## Accessing the GPU

In [ ]:
pipeline.to("cuda")

## Giving a text prompt and asking the model to generate an image from Gaussian noise

In [ ]:
image = pipeline("An image of a squirrel in Picasso style").images[0]

In [ ]:
image

In [ ]:
image.save("image_of_squirrel_painting.png")

## Using a different scheduler

In [ ]:
from diffusers import EulerDiscreteScheduler

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)
pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config)

In [ ]:
image = pipeline("An image of a squirrel in Picasso style").images[0]

In [ ]:
image

## Lets explore a model trained on cat images

In [ ]:
from diffusers import UNet2DModel

In [ ]:
repo_id = "google/ddpm-cat-256"

model = UNet2DModel.from_pretrained(repo_id, use_safetensors=True)

### Model parameters

In [ ]:
model.config

### Creating an image shape with random gaussian noise

In [ ]:
import torch

torch.manual_seed(0)

noisy_sample = torch.randn(1, model.config.in_channels, model.config.sample_size, model.config.sample_size)

noisy_sample.shape
torch.Size([1, 3, 256, 256])

### Inference

In [ ]:
with torch.no_grad():

    noisy_residual = model(sample=noisy_sample, timestep=2).sample

## Coupling a model with a scheduler

### Importing a scheduler

In [ ]:
from diffusers import DDPMScheduler

In [ ]:
scheduler = DDPMScheduler.from_pretrained(repo_id)

scheduler

### Lets generate a slightly less noisy sample

In [ ]:
less_noisy_sample = scheduler.step(model_output=noisy_residual, timestep=2, sample=noisy_sample).prev_sample

In [ ]:
less_noisy_sample.shape

### Visualization

In [ ]:
import PIL.Image
import numpy as np

In [ ]:
def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)
    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

Moving the model and noisy sample to gpu

In [ ]:
model.to("cuda")

In [ ]:
noisy_sample = noisy_sample.to("cuda")

In [ ]:
import tqdm

In [ ]:
sample = noisy_sample

In [ ]:
for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):

    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample
    # 2. compute less noisy image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample
    # 3. optionally look at image
    if (i + 1) % 50 == 0:
        display_sample(sample, i + 1)